
# Getting started

Once you've chosen your scenario, download the data from the Iowa website in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [1]:
import pandas as pd

import numpy as np
## Load the data into a DataFrame
liquor = pd.read_csv('../../../../Iowa_Liquor_Sales_reduced.csv')[['Store Number',
               'Date',
               'City',
               'Zip Code',
               'County',
               'Bottle Volume (ml)', 
               'State Bottle Cost', 
               'State Bottle Retail', 
               'Bottles Sold', 
               'Sale (Dollars)', 
               'Volume Sold (Liters)', 
               'Volume Sold (Gallons)']]


//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
## Make necessary column modifications

## Transform the dates
liquor["Date"] = pd.to_datetime(liquor["Date"], format="%m/%d/%Y")
liquor['Year'] = liquor['Date'].dt.year
liquor['Month'] = liquor['Date'].dt.month

## Reformat column names to replace spaces with underscores
liquor.columns = [c.replace(' ', '_') for c in liquor.columns]
liquor.columns = [c.replace(')', '') for c in liquor.columns]
liquor.columns = [c.replace('(', '') for c in liquor.columns]

## turn to lowercase because I hate initcap
liquor.columns = [c.lower() for c in liquor.columns]

In [3]:
## capitalize city and county for consistency
liquor.city = liquor.city.str.upper()
liquor.county = liquor.county.str.upper()

## remove dollar signs and convert to floats
liquor['sale_dollars'] = liquor['sale_dollars'].str.replace('$','').astype(np.float64)
liquor['state_bottle_retail'] = liquor['state_bottle_retail'].str.replace('$','').astype(np.float64)
liquor['state_bottle_cost'] = liquor['state_bottle_cost'].str.replace('$','').astype(np.float64)

In [4]:
## check: missing values, dtypes, df shape, describe, unique values, duplicates

def eda(dataframe):
    
    print "missing values \n", dataframe.isnull().sum()## count number of null values per column
    print ''
    print "dataframe types \n", dataframe.dtypes       ## list data type of each column
    print ''
    print "dataframe shape \n", dataframe.shape        ## rows by columns
    print ''
    print "dataframe describe \n", dataframe.describe()## stats -- mean, min, max, etc.. 
    print ''
    for item in dataframe:                             ## count number of unique values per column
        print item, '= ', dataframe[item].nunique()
    print ''
    print 'num duplicates:', dataframe.duplicated().sum()

# eda(liquor)

In [5]:
## drop duplicate rows
liquor.drop_duplicates(inplace=True)
print 'num duplicates:', liquor.duplicated().sum()

## look into missing county_numbers and counties
county_lookup = liquor[['zip_code','county','county_number']][liquor.county.isnull()==False]
county_lookup.drop_duplicates(inplace=True)

num duplicates: 0


KeyError: "['county_number'] not in index"

In [ ]:
## grab a list of zip codes for iowa to help fill in missing county names
iowa_zips = pd.read_csv('../../../../iowa_zips.csv')
iowa_zips.columns = [c.replace(' ', '_') for c in iowa_zips.columns]

In [ ]:
# set up connection to SQLalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

# push dfs into sql tables
liquor.to_sql('liquor', engine)
county_lookup.to_sql('county_lookup', engine)
iowa_zips.to_sql('iowa_zips', engine)

In [ ]:
# run updates to fill in county names
conn.execute("update liquor set county = (select upper(county) from county_lookup where county_lookup.zip_code = liquor.zip_code) where county is null;")
conn.execute("update liquor set county = (select upper(County) from iowa_zips where iowa_zips.Zip_Code = liquor.zip_code) where county is null;")
# run updates to fill in county_numbers
conn.execute("update liquor set county_number = (select county_number from county_lookup where county_lookup.zip_code = liquor.zip_code and county_lookup.county = liquor.county) where county_number is null;")
# conn.execute("update liquor set quarter = (case when year=2015 and month<=3 then '1Q15' when year=2015 and month<=6 then '2Q15' when year=2015 and month<=9 then '3Q15' when year=2015 and month<=12 then '4Q15' else '1Q16' end);")
# pull sql table into a pandas df
liquor_sql = pd.read_sql_table('liquor',engine)
# close sql connection
conn.close()

In [ ]:
# check missing values for counties
eda(liquor_sql)

liquor_sql['zip_code'][liquor_sql.county.isnull()==True]
liquor = liquor_sql

In [ ]:
## now that all counties are filled in, export data to be used for mapping in tableau
liquor.to_csv('../../../../Iowa_Liquor_sales_sample_10pct_CLEAN.csv') 


## because i'm neurotic, found coordinates for counties that couldn't be mapped in tableau
# https://dom.iowa.gov/document/iowa-county-codes-and-coordinates

# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.scatter(liquor.bottles_sold,liquor.sale_dollars)
plt.xlabel('Bottles Sold')
plt.ylabel('Sale Amount $')
plt.title('Obvious Relationship between Bottles Sold and Sale Amount')
plt.show()

plt.scatter(liquor.volume_sold_liters,liquor.sale_dollars)
plt.show()

plt.scatter(liquor.bottles_sold,liquor.volume_sold_liters)
plt.show()

plt.scatter(liquor.bottles_sold,liquor.state_bottle_retail)
plt.show()


# Record your findings

Be sure to write out any observations from your exploratory analysis.


# Mine the data

Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations here for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.
 


In [ ]:
by_store = liquor[['store_number',
                   'quarter',
                   'sale_dollars',
                   'volume_sold_liters',
                   'bottles_sold']].groupby(['store_number',
                                             'quarter']).agg({'sale_dollars': [np.sum, np.mean],
                                                            'volume_sold_liters': [np.sum, np.mean],
                                                            'bottles_sold' : [np.sum, np.mean]
                                                           
                                                           })
by_county = liquor[['county',
                   'quarter',
                   'sale_dollars',
                   'volume_sold_liters',
                   'bottles_sold']].groupby(['county',
                                             'quarter']).agg({'sale_dollars': [np.sum, np.mean],
                                                            'volume_sold_liters': [np.sum, np.mean],
                                                            'bottles_sold' : [np.sum, np.mean]
                                                           })

by_city = liquor[['city',
                   'quarter',
                   'sale_dollars',
                   'volume_sold_liters',
                   'bottles_sold']].groupby(['city',
                                             'quarter']).agg({'sale_dollars': [np.sum, np.mean],
                                                            'volume_sold_liters': [np.sum, np.mean],
                                                            'bottles_sold' : [np.sum, np.mean]
                                                           })

In [ ]:
sns.pairplot(by_store)

In [ ]:
sns.pairplot(by_county)

In [ ]:
sns.pairplot(by_city)


# Refine the data

Look for any statistical relationships, correlations, or other relevant properties of the dataset.

In [ ]:
liquor_by_store = liquor[['store_number',
                          'sale_dollars',
                          'bottles_sold', 
                          'state_bottle_retail']].groupby(liquor.store_number).agg({'sale_dollars' : np.sum,
                                                                                    'bottles_sold' : np.sum,
                                                                                    'state_bottle_retail' : np.sum})

liquor_by_zip = liquor[['zip_code',
                          'sale_dollars',
                          'bottles_sold', 
                          'state_bottle_retail',
                        'store_number']].groupby(liquor.zip_code).agg({'sale_dollars' : np.sum,
                                                                                    'bottles_sold' : np.sum,
                                                                                    'state_bottle_retail' : np.sum,
                                                                                   'store_number' : lambda x: x.nunique()})
liquor_by_city = liquor[['city',
                          'sale_dollars',
                          'bottles_sold', 
                          'state_bottle_retail',
                        'store_number']].groupby(liquor.city).agg({'sale_dollars' : np.sum,
                                                                                    'bottles_sold' : np.sum,
                                                                                    'state_bottle_retail' : np.sum,
                                                                           'store_number' : lambda x: x.nunique()})
liquor_by_county = liquor[['county',
                          'sale_dollars',
                          'bottles_sold', 
                          'state_bottle_retail',
                          'store_number']].groupby(liquor.county).agg({'sale_dollars' : np.sum,
                                                                                    'bottles_sold' : np.sum,
                                                                                    'state_bottle_retail' : np.sum,
                                                                             'store_number' : lambda x: x.nunique()})


liquor_by_store['avg_price_per_bottle'] = liquor_by_store.sale_dollars / liquor_by_store.bottles_sold
liquor_by_zip['avg_price_per_bottle'] = liquor_by_zip.sale_dollars / liquor_by_zip.bottles_sold
liquor_by_city['avg_price_per_bottle'] = liquor_by_city.sale_dollars / liquor_by_city.bottles_sold
liquor_by_county['avg_price_per_bottle'] = liquor_by_county.sale_dollars / liquor_by_county.bottles_sold

liquor_by_zip['avg_sales_store'] = liquor_by_zip.sale_dollars / (liquor_by_zip.bottles_sold)
liquor_by_city['avg_sales_store'] = liquor_by_city.sale_dollars / (liquor_by_city.bottles_sold)
liquor_by_county['avg_sales_store'] = liquor_by_county.sale_dollars / (liquor_by_county.bottles_sold)

liquor_by_zip['avg_sales_store_plus1'] = liquor_by_zip.sale_dollars / (liquor_by_zip.bottles_sold + 1)
liquor_by_city['avg_sales_store_plus1'] = liquor_by_city.sale_dollars / (liquor_by_city.bottles_sold + 1)
liquor_by_county['avg_sales_store_plus1'] = liquor_by_county.sale_dollars / (liquor_by_county.bottles_sold + 1)

liquor_by_zip['delta_sales'] = liquor_by_zip.avg_sales_store - liquor_by_zip.avg_sales_store_plus1
liquor_by_city['delta_sales'] = liquor_by_city.avg_sales_store - liquor_by_city.avg_sales_store_plus1
liquor_by_county['delta_sales'] = liquor_by_county.avg_sales_store - liquor_by_county.avg_sales_store_plus1

In [ ]:
sns.pairplot(liquor_by_store)

In [ ]:
sns.pairplot(liquor_by_zip)

In [ ]:
sns.pairplot(liquor_by_city)

In [ ]:
sns.pairplot(liquor_by_county)

# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

In [ ]:
import statsmodels.formula.api as sm #import statsmodels

In [ ]:

X = liquor_desmoines[['bottles_sold','state_bottle_retail']]
y = liquor_desmoines['sale_dollars']

lm = linear_model.LassoCV(cv=5)
model_desmoines = lm.fit(X,y)
print model_desmoines.score(X,y)
print model_desmoines.alpha_

In [ ]:
from sklearn import linear_model
from sklearn.metrics import r2_score


liquor_by_zip.sort_values('sale_dollars',ascending=False)
liquor_50314 = liquor[['store_number',
                           'bottles_sold', 
                           'state_bottle_retail', 
                           'sale_dollars']][liquor.zip_code=='50314'].groupby(liquor.store_number).agg({'bottles_sold': np.sum,
                                                                                                         'state_bottle_retail' : np.sum, 
                                                                                                         'sale_dollars' : np.sum})
X = liquor_50314[['bottles_sold','state_bottle_retail']]
y = liquor_50314['sale_dollars']

lm = linear_model.LassoCV(cv=5)
model_50314 = lm.fit(X,y)
print '50314 model score = ', model_50314.score(X,y)
print '50314 alpha =', model_50314.alpha_
print ''


liquor_50320 = liquor[['store_number',
                           'bottles_sold', 
                           'state_bottle_retail', 
                           'sale_dollars']][liquor.zip_code=='50320'].groupby(liquor.store_number).agg({'bottles_sold': np.sum,
                                                                                                         'state_bottle_retail' : np.sum, 
                                                                                                         'sale_dollars' : np.sum})
                                                                                                         
X = liquor_50320[['bottles_sold','state_bottle_retail']]
y = liquor_50320['sale_dollars']

lm = linear_model.LassoCV(cv=5)
model_50320 = lm.fit(X,y)
print '50320 model score =', model_50320.score(X,y)
print '50320 alpha =', model_50320.alpha_
print ''


liquor_desmoines = liquor[['store_number',
                           'bottles_sold', 
                           'state_bottle_retail', 
                           'sale_dollars']][liquor.city=='DES MOINES'].groupby(liquor.store_number).agg({'bottles_sold': np.sum,
                                                                                                         'state_bottle_retail' : np.sum, 
                                                                                                         'sale_dollars' : np.sum})
X = liquor_desmoines[['bottles_sold','state_bottle_retail']]
y = liquor_desmoines['sale_dollars']

lm = linear_model.LassoCV(cv=5)
model_desmoines = lm.fit(X,y)
print 'Des Moines model score = ', model_desmoines.score(X,y)
print 'Des Moines alpha = ', model_desmoines.alpha_

In [ ]:
X = liquor_by_store[['bottles_sold','state_bottle_retail']]
y = liquor_by_store['sale_dollars']
lm = linear_model.LassoCV(cv=5)
model = lm.fit(X,y)
print model.score(X,y)
print model.alpha_
print ''
liquor_by_store['predictions'] = model.predict(X)
print 'TOP PERFORMING STORES'
liquor_by_store[['sale_dollars','predictions']].sort_values('predictions', ascending=False)[0:20]

# Plot your results

Again make sure that you record any valuable information. For example, in the tax scenario, did you find the sales from the first three months of the year to be a good predictor of the total sales for the year? Plot the predictions versus the true values and discuss the successes and limitations of your models

In [ ]:
predictions = model_desmoines.predict(X)

plt.scatter(y, predictions)
plt.ylabel("Predictions")
plt.xlabel("True Values")
plt.show()

In [ ]:
predictions = model_50314.predict(X)

plt.scatter(y, predictions)
plt.ylabel("Predictions")
plt.xlabel("True Values")
plt.show()


predictions = model_50320.predict(X)

plt.scatter(y, predictions)
plt.ylabel("Predictions")
plt.xlabel("True Values")
plt.show()

# Present the Results

Present your conclusions and results. If you have more than one interesting model feel free to include more than one along with a discussion. Use your work in this notebook to prepare your write-up.